# Training neural networks

When trying to train a neural networks, many problems awaits us:
- Gradient growing ever smaller or larger during the backward pass making lower layers not converging.
- Not enough data or the data is not labelled sufficiently
- Slow training
- Overfitting by having too many parameters

In this chapter we will discuss about solutions to these problems and also about regularization techniques for neural networks.

## The vanishing/exploding gradient problem

As we know backpropagation works by first performing a forward pass through the layers of the neural network to compute the model's output and evaluate the
loss. Then, it performs a backward pass, where the error gradient is propagated backward through the network to determine how much each parameter (weights 
and biases) contributed to the error. This information is used to compute the gradient of the cost function with respect to each parameter, which is then 
used to update the parameters and minimize the loss. But the problem is that as the algorithm progresses backward through the layers during 
backpropagation, the gradients can sometimes become smaller and smaller. When this happens, the gradients in the lower layers (closer to the input) may 
become so small that their corresponding parameters barely change, preventing the model from learning effectively. This issue is known as the 
_vanishing gradient_ problem. The opposite can also occur, where the gradients grow larger as we move backward through the layers, causing the parameters 
to receive very large updates. This can lead to the model diverging during training, a situation referred to as the _exploding gradient_ problem. It has
been found that the problem comes from a certain combination of activation function and weight initialization technique(the _sigmoid function_ is the
activation function being referred to here and the initialization technique is the normal distribution).  
One solution presented by scientist is to ensure that the gradient have to be of equal variance before and after flowing through each layer(in the backward
pass) although it is not possible unless the layer has an equal number of inputs(that we will call $fan_{in}$) and outputs(that we will call $fan_{out}$). A
compromise is to initiliaze the connections weight randomly such as the distribution follows this technique(where the activation is the sigmoid function):  
Normal distribution of mean 0 and variance
$$\sigma^2 = \frac{1}{fan_{avg}} $$ 
or a uniform distribution between -r and r where 
$$r = \sqrt{\frac{3}{fan_{avg}}} $$
Where
$$fan_{avg} = (fan_{in} + fan_{out}) / 2 $$

This initialization technique is called the __Xavier initialization__ or __Glorot initialization__, it considerably speed up training. And we replace 
$fan_{avg}$ by $fan_{in}$ in the equation we get the __LeCun initialization__ algorithm. The initialization algorithm to use if we want to use ReLU as
activation is called _He_ initialization. By default keras uses glorot initialization with a uniform distribution but we change this behavior like this:

In [1]:
import tensorflow as tf

dense = tf.keras.layers.Dense(50, activation="relu", kernel_initializer="he_normal")

Another problem that neural network models suffer is the _dying ReLU_ problem. During training, some neurons "dies" meaning they start only outputing 0.
This happens when the weights are tweaked so much that the inputs of the ReLU function(the sum of the neurons input + a bias term) output a negative value
for all instances in the training set. For that it is advisable to use other amelioration functions that are variants of ReLU.

### The LeakyReLU activation function

This function is defined as follow:
$$LeakyReLu_{\alpha}(z) = max(\alpha z, z) $$
The $\alpha$ parameter defines how much the function "leaks", in other words it is the slope of the function when z < 0. Keras include this option in
its layer class:

In [ ]:
leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2) # This value leads to better performance than a lower value
dense = tf.keras.layers.Dense(30, activation=leaky_relu, kernel_initializer="he_normal")

## ELU, SELU, GELU, Swish and Mish activation functions

A more performant activation function that have been found by scientist is the ELU(exponential linear unit) and is defined by the following equation:
$$\text{ELU}_\alpha(z) = \begin{cases}
\alpha(\exp(z) - 1) & \text{if } z < 0 \\
z & \text{if } z \geq 0
\end{cases}$$
We should note that it is important to use _He_ initialization(the same goes for any other ReLU variant) and that at test time this function will be slower
than ReLU.  
Another variant of the previous one is SELU(Scaled Exponiental Linear Unit), if we build a model with all the hidden layers using SELU they will eventually self-normalize. Note that for this function to work, the inputs must be standardized with mean 0 and standard deviation of 1, we must use LeCun
initialization.  
If we want more accurate activation function but at the cost of a heavier computation time we can look to GELU defined like this:
$$\text{GELU}(z) = z\phi (z) $$ 
Where $\phi (z)$ is the probability that a randomly sampled value from a normal distribution of mean 0 and variance 1 is lower than _z_. Other activation
functions avalaible are Swish:
$$\text{Swish}_{\beta}(z) = z\sigma(\beta z) $$
Where $\beta$ is a scaling hyperparameter(we can pass it to gradient descent to determine the best value to use). The last one we can use is the Mish
function:
$$\text{mish}(z) = z\tanh(softplus(z)) $$
Note that keras does not support mish natively but we can implement it and use it as a custom activation function.

### Batch Normalization

A solution to solve unstable gradients other than changing activation function is to use something called _batch normalization_. It consists of adding an
operation before or after the activation function of each layer. This operation simply zero centers and shift the result using 2 new parameters one for
scaling and one for shifting. In order to do that the algorithm need to estimate each input means and standard deviation. Here are the steps the algorithm
follows:
$$\mu _B = \frac{1}{m_B}\sum_{i = 1}^{m_B}x^{(i)} $$
$$\sigma _B^2 = \frac{1}{m_B}\sum_{i = 1}^{m_B}(x^{(i)} - \mu _B)^2$$
$$\hat{x}^{(i)} = \frac{x^{(i)} - \mu _B}{\sqrt{\mu _B^2 + \epsilon}} $$
$$\text{z}^{(i)} = \gamma ⊗ \hat{x}^{(i)} + \beta $$
Where:
- $\mu _B$ is the vector of all the input means evaluated over the whole the mini-batch B(one mean per input).
- $m_B$ is the number of instances in the mini-batch.
- $\sigma _B$ is the vector of input standard deviation.
- $\text{x}^{(i)}$ is the vector of zero centered and normalized inputs for the instance _i_.
- $\epsilon$ is the _smoothing term_. It ensure that the value of the gradient don't grow too large.
- $\gamma$ is the output scale parameter vector(one for each output).
- ⊗ is the element wise multiplication(in a vector where all the element are multiplied with their corresponding value).
- $\beta$ is the output shift(the offset) vector. Each output is shifted by its corresponding shift value.
- $\text{z}^{(i)}$ is the output of the algorithm. It is the scaled and shifted version of each input.

The keras implementation is the following:

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(), # Making it a layer work normally well.
    tf.keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax")
])

The default hyperparameters can effectively be tweaked but sometime we can add another one called _momentum_. This hyperparameter is used by the algorithm
when it updates the exponential, given a new value v (i.e., a new vector of input means or standard deviations computed over the current batch), the layer 
updates the running average $\hat{v}$ using the following equation:
$$\hat{v} \gets \hat{v} * \text{momentum} + v * (1 - \text{momentum})$$

### Regularization through gradient clipping

Another technique for dealing with gradient explosion is to clip a gradient before it exceed a threshold. We can set it up in keras like this:

In [ ]:
optimizer = tf.keras.optimizer.SGD(clipvalue=1.0)# All the gradient component values will be clamped between -1 and 1
# Then we compile our model using this optimizer

## Transfer learning

When we need to train very large DNN, it is advisable to try to find existing pretrained model that accomplish a similar task. If we find such model,
we can reuse most of its layer(not the top ones though). Let's try with the fashion MNIST dataset.

In [ ]:
# For this example let's assume that we have created and trained a model A
model_A = tf.keras.models.load_model("model_A")
model_B_on_A = tf.keras.Sequential(model_A.layers[-1])
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))
# But this method of doing it training model_B_on_A affect model_A if we want to avoid that we have to clone model_A first
model_A_clone = tf.keras.model.clone_model(model_A)
# And it weights
model_A_clone.set_weights(model_A.get_weights())

Now we could train model\_B\_on\_A for task B, but since the new output layer was initialized randomly it will make large errors (at least during the
first few epochs), so there will be large error gradients that may wreck the reused weights. To avoid this, one approach is to freeze the reused layers
during the first few epochs, giving the new layer some time to learn reasonable weights.

In [ ]:
for layer in model_B_on_A.layers[-1]:
    layer.trainable = False

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

Now we can unfreeze the reused layers to finetune them.

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))
for layer in model_B_on_A.layers[-1]:
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"]) # We need to recompile the model
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))

### Unsupervised learning

If we do not have sufficiently labelled data is train layers on an auxiliary task in which we can easily obtain labeled instances and reuse this layers for
our model.

### Improving performance through faster optimizers

While gradient descent updates the weights θ by directly subtracting the gradient of the cost function J(θ) with regard to the weights (∇ J(θ))
multiplied by the learning rate η. The equation being:
$$\theta \gets \theta - \eta ∇_{\theta}J(\theta)$$
It does not care about the previous gradient. This is the problem that the _momentum_ algorithm proposes to solve. At each iteration it substract the local
gradient from the _momentum vector_(multiplied by the learning rate $\eta$). Additionaly the algorithm introduces a hyperparameter $\beta$(called the 
momentum) to control the value of the momentum from growing too large(the value of this hyperparameter must be set between 0 and 1, a common value being
0.9). The algorithm it uses is the following:
$$\text{m} \gets \beta m - \eta ∇_{\theta} J(\theta)$$
$$\theta \gets \theta + \text{m}$$
Implementing it in keras look like this:

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

A variant of momentum is __Nesterov Accelerated Gradient__(NAG) method. It measures the gradient of the cost function not at the local position θ but 
slightly ahead in the direction of the momentum, at θ + βm:
$$\text{m} \gets \beta m - \eta ∇_{\theta} J(\theta + \beta m)$$
$$\theta \gets \theta + m $$ 
Note that ∇ is the gradient measured at a given moment. Using it in keras is done like this:

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

Another gradient variant is meant to correct the fact that gradient descent starts by quickly going down the steepest slope, which does not point straight 
toward the global optimum, then it very slowly goes down to the bottom of the valley. The algorithm should correct its direction earlier to point a bit
more toward the global optimum. The __AdaGrad algorithm__ achieves this correction by scaling down the gradient vector along the steepest dimensions:
$$\text{s} \gets \text{s} + ∇_{\theta} J(\theta) ⊗ ∇_{\theta} J(\theta) $$
$$\theta \gets \theta - \eta∇_{\theta} J(\theta) ⊘ \sqrt{s + \epsilon} $$
AdaGrad frequently performs well for simple quadratic problems, but it often stops too early when training neural networks: the learning rate gets scaled
down so much that the algorithm ends up stopping entirely before reaching the global optimum. So even though Keras has an Adagrad optimizer, we should not 
use it to train deep neural networks (it may be efficient for simpler tasks such as linear regression). Still other optimizers built on it so it might
be helpful to know. An algorithm called __RMSProp__ attempt to solve AdaGrad's problem by accumulating only the gradients from the most recent iterations
instead of all the gradients since the beggining of training:
$$\text{s} \gets \text{s}\rho + (1 - \rho)∇_{\theta} J(\theta) ⊗ ∇_{\theta}J(\theta) $$
$$\theta \gets \theta - \eta∇_{\theta} ⊘ \sqrt{s + \epsilon}$$
$\rho$ is called the _decay rate_ and is usually set to 0.9.

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

__Adam__, which stands for adaptive moment estimation, combines the ideas of momentum optimization and RMSProp: just like momentum optimization, it keeps 
track of an exponentially decaying average of past gradients; and just like RMSProp, it keeps track of an exponentially decaying average of past squared 
gradients. These are estimations of the mean and (uncentered) variance of the gradients. The mean is often called the _first moment_ while the variance is 
often called the _second moment_, hence the name of the algorithm.: 
$$\text{m} \gets \beta_{1} m - (1 - \beta_{1}) ∇_{\theta}J(\theta)$$
$$\text{s} \gets \beta_{2}s + (1 - \beta_{2}) ∇_{\theta}J(\theta) ⊗ ∇_{\theta}J(\theta)$$
$$\hat{m} \gets \frac{m}{1 - \beta_{1}^{t}} $$
$$\hat{s} \gets \frac{s}{1 - \beta_{2}^{t}} $$
$$\theta \gets \theta + \eta \hat{m} ⊘ \sqrt{\hat{s} + \epsilon} $$
_t_ represents the iteration number starting from 1. $\beta_{1}$ corresponds to $\beta$ in the momentum optimization algorithm and $\beta_{2}$ corresponds
to $\rho$ in RMSProp.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

### Regularization 

To regularize our model we have many tools at our disposal. First we can talk about the _Dropout_ which at each training step every neuron(excluding the
output neurons) has a probability _p_ of being _dropped out_ meaning it will be entirely ignored during this training step. The hyperparameter _p_ is called
the dropout rate vary mostly from 20% to 50%. At the end we get a more robust network. Suppose p = 75%: on average only 25% of all neurons are active at 
each step during training. This means that after training, a neuron would be connected to four times as many input neurons as it would be during training. 
To compensate for this fact, we need to multiply each neuron’s input connection weights by four during training. If we don’t, the neural network will not 
perform well as it will see different data during and after training. More generally, we need to divide the connection weights by the keep probability 
(1 – p) during training.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(10, activation="softmax")
])

We can also use something known as the _Monte Carlo dropout_ which can boost the performance of any trained dropout model without having to retrain it or 
even modify it at all. It also provides a much better measure of the model’s uncertainty, and it can be implemented in just a few lines of code. 
Another popular regularization technique for neural networks is called max-norm regularization: for each neuron, it constrains the weights w of the 30
incoming connections such that $∥ w ∥_2$ ≤ r, where r is the max-norm hyperparameter and ∥ · ∥ is the $ℓ_1$ norm.